# Covid-19 Detection

## Pablo Lázaro Herrasti and Rubén Barco Terrones

### Import

In [16]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

#### Directories

In [5]:
dir_covid = 'D:/Covid-19 Data/COVID-19 Radiography Database/'
dir_covid_images = dir_covid + 'COVID-19/'
dir_normal_images = dir_covid + 'NORMAL/'
dir_pneumonia_images = dir_covid + 'Viral Pneumonia/'
all_dir_images = [dir_normal_images, dir_covid_images, dir_pneumonia_images]
dir_covid_metadata = dir_covid + 'COVID-19.metadata.xlsx'
dir_normal_metadata = dir_covid + 'NORMAL.metadata.xlsx'
dir_pneumonia_metadata = dir_covid + 'Viral Pneumonia.matadata.xlsx'

#### Reading and preprocessing data

In [6]:
metadata_covid = pd.read_excel(dir_covid_metadata)
metadata_covid['label'] = 0
metadata_normal = pd.read_excel(dir_normal_metadata)
metadata_normal['label'] = 1
metadata_pneumonia = pd.read_excel(dir_pneumonia_metadata)
metadata_pneumonia['label'] = 2
metadata_all = {dir_covid_images:metadata_covid, dir_normal_images:metadata_normal, dir_pneumonia_images:metadata_pneumonia}

In [7]:
#Reading Image data and converting it into pixels and separating class labels
Data=[]
Label=[]
directory='D:/Data_Master/Deep Learning/ChestXRay2017/All'

for dir_images in all_dir_images:
    files = os.listdir(dir_images)
    for index, row in metadata_all[dir_images].iterrows():
        Label.append(row['label'])
        filename=os.path.join(dir_images, files[index])
        im=image.load_img(filename,target_size=(224, 224))
        im=np.reshape(im,(224,224,3))
        im=im.astype('float32') / 255
        Data.append(im)  

In [8]:
#Train Test Split
X_train, X_1, y_train, y_1 = train_test_split(np.array(Data), np.array(Label), test_size=0.2, random_state=42,stratify=Label)

#Train Test Split
X_cv, X_test, y_cv, y_test = train_test_split(X_1, y_1, test_size=0.2, random_state=42,stratify=y_1)

In [9]:
img_width=224
img_height=224

from keras import backend as K

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
    X_train=X_train.reshape(X_train.shape[0],3,img_width,img_height)
    X_cv=X_cv.reshape(X_cv.shape[0],3,img_width,img_height)
    X_test=X_test.reshape(X_test.shape[0],3,img_width,img_height)
    
else:
    input_shape = (img_width, img_height, 3)
    X_train=X_train.reshape(X_train.shape[0],img_width,img_height,3)
    X_cv=X_cv.reshape(X_cv.shape[0],img_width,img_height,3)
    X_test=X_test.reshape(X_test.shape[0],img_width,img_height,3)
    
del Data

Using TensorFlow backend.


#### Architecture for binary classification

In [12]:
model=Sequential()
model.add(Conv2D(32, 3, input_shape=input_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(2))
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(2))
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
__________

#### Training model

In [13]:
n_epochs = 4
batch_size = 32
model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=n_epochs, validation_data=(X_cv,y_cv))

Train on 2324 samples, validate on 464 samples
Epoch 1/4
2324/2324 [==============================] - 146s 63ms/sample - loss: 0.6795 - acc: 0.7207 - val_loss: 0.3495 - val_acc: 0.8793
Epoch 2/4
2324/2324 [==============================] - 136s 59ms/sample - loss: 0.2772 - acc: 0.9015 - val_loss: 0.1585 - val_acc: 0.9397
Epoch 3/4
2324/2324 [==============================] - 137s 59ms/sample - loss: 0.1796 - acc: 0.9398 - val_loss: 0.1257 - val_acc: 0.9526
Epoch 4/4
2324/2324 [==============================] - 137s 59ms/sample - loss: 0.1229 - acc: 0.9583 - val_loss: 0.1321 - val_acc: 0.9461


### General accuracy

##### Multiclass prediction

In [19]:
train_acc = accuracy_score(model.predict_classes(X_train), y_train)
valid_acc = accuracy_score(model.predict_classes(X_cv), y_cv)
test_acc = accuracy_score(model.predict_classes(X_test), y_test)
print("The final train accuracy is ",train_acc*100,"%")
print("The final validation accuracy is ",valid_acc*100,"%")
print("The final test accuracy is ",test_acc*100,"%")

The final train accuracy is  96.90189328743546 %
The final validation accuracy is  94.61206896551724 %
The final test accuracy is  94.87179487179486 %


##### Binary prediction

In [ ]:
train_acc = accuracy_score((model.predict(X_train)>0.5).astype(int), y_train)
valid_acc = accuracy_score((model.predict(X_cv)>0.5).astype(int), y_cv)
test_acc = accuracy_score((model.predict(X_test)>0.5).astype(int), y_test)
print("The final train accuracy is ",train_acc*100,"%")
print("The final validation accuracy is ",valid_acc*100,"%")
print("The final test accuracy is ",test_acc*100,"%")

#### Covid accuracy

In [23]:
X_covid = X_test[y_test == 0]
y_covid = y_test[y_test == 0]
test_acc = accuracy_score(model.predict_classes(X_covid), y_covid)
print("The final test accuracy for COVID-19 is ",test_acc*100,"%")

The final test accuracy for COVID-19 is  98.14814814814815 %


In [24]:
X_normal = X_test[y_test == 1]
y_normal = y_test[y_test == 1]
test_acc = accuracy_score(model.predict_classes(X_normal), y_normal)
print("The final test accuracy for NORMAL is ",test_acc*100,"%")

The final test accuracy for NORMAL is  98.14814814814815 %


In [26]:
X_pneumonia = X_test[y_test == 2]
y_pneumonia = y_test[y_test == 2]
test_acc = accuracy_score(model.predict_classes(X_pneumonia), y_pneumonia)
print("The final test accuracy for PNEUMONIA is ",test_acc*100,"%")

The final test accuracy for PNEUMONIA is  92.5925925925926 %
